In [16]:
from werkzeug.wrappers import Request, Response
from flask import Flask,render_template,send_file,make_response,request
import json
import pandas as pd

import numpy as np
from sklearn import ensemble, model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from lime.lime_tabular import LimeTabularExplainer

app = Flask(__name__)
print(dir(json))

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
dataset  = pd.read_csv(url, names=['sepal length', 'sepal width', 'petal length', 'petal width','target'])


def index_containing_substring(the_list, substring):
    for i, s in enumerate(the_list):
        if substring in s:
              return i
    return -1

def reduce_dim(df):
    variables = ['sepal length', 'sepal width', 'petal length', 'petal width']
    x = df.loc[:, variables].values
    y = df.loc[:,['target']].values
    x = StandardScaler().fit_transform(x)
    x = pd.DataFrame(x)
    pca = PCA()
    x_pca = np.array(pca.fit_transform(x))
    display(pca.explained_variance_ratio_)
    return pd.DataFrame(x_pca[:,0:2],columns=['x','y'])

@app.route('/box')
def boxplot():
    ret = pd.DataFrame({'x':np.random.randint(-1000, high=1000, size=1000),
                        'y':np.random.randint(-1, high=2000, size=1000),
                        'z':np.random.randint(-1000, high=2000, size=1000),
                       })
    ret=ret.set_index('x')
    resp = make_response(ret.to_csv())
    resp.headers["Content-Disposition"] = "attachment; filename=export.csv"
    resp.headers["Content-Type"] = "text/csv"
    return resp

@app.route('/datasets_entries')
def dataset_entries():
    return json.dumps([['dataset1','entry_count = 10'],['dataset2'],['dataset3','entry_count = 100']])


@app.route('//dataset_<dataset>_details')
def dataset_details(dataset):
    return json.dumps(['dataset name','dataset entry1','dataset entry2','dataset entry3'])

@app.route('/favicon.ico')
def icon():
    return None

@app.route('/doc_indx_<docs>')
def get_documents(docs):
    return [1,2,3,4,5]

@app.route('/')
def route():
    return render_template('index.html')

@app.route('/get_dataset_<dataset>')
def load_dataset(dataset):
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
    df = pd.read_csv(url, names=['sepal length', 'sepal width', 'petal length', 'petal width','target'])

    df = reduce_dim(df)
    df['label'] = 1
    df['color'] = 'blue'

    
    display(df)
    return df.to_json()

@app.route('/elaborate_dataset')
def evaluate():
    j=json.loads(request.args.get('dataset'))
    data = pd.DataFrame(j)
    #dataset['target'] = j
    data = dataset.select_dtypes(['number'])
    display(data)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(
    data, j)

    regressor = ensemble.RandomForestRegressor()
    regressor.fit(X_train, y_train)
    
    display(X_train)
    display(regressor.predict(X_test))
    explainer = LimeTabularExplainer(X_train.to_numpy(),
                                     training_labels =['sepal length', 'sepal width', 'petal length', 'petal width'],
                                     feature_names=['sepal length', 'sepal width', 'petal length', 'petal width'],
                                     mode='regression')

    X_test = X_test.to_numpy()
    # Now explain a prediction
    exp = explainer.explain_instance(X_test[25], regressor.predict,
            num_features=4)
    
    exp.save_to_file("templates/explanation_test.html")
    
    f = open("templates/explanation_test.html", "r",encoding="utf8")
    contents = f.readlines()
    f.close()
    
    index = index_containing_substring(contents, ".templateSettings.interpolate ")
    del contents[index]
    
    f = open("templates/explanation_test.html", "w",encoding="utf8")
    contents = "".join(contents)
    f.write(contents)
    f.close()
    
    evaluation = {'src':'explanation_test'}
    return json.dumps(evaluation).encode("utf8")

@app.route('/<page>')
def rout(page):
    return render_template(page+'.html')

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9001, app,reloader_interval = 5000)

['JSONDecodeError', 'JSONDecoder', 'JSONEncoder', '__all__', '__author__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_default_decoder', '_default_encoder', 'codecs', 'decoder', 'detect_encoding', 'dump', 'dumps', 'encoder', 'load', 'loads', 'scanner']


 * Running on http://localhost:9001/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Mar/2021 16:59:00] "GET /explanation?performances=1,0.5,1,0.5&other=true HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2021 16:59:00] "GET /static/Histogram.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Mar/2021 16:59:00] "GET /static/Boxplot.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Mar/2021 16:59:00] "GET /static/Performance_wheel.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Mar/2021 16:59:01] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2021 16:59:02] "GET /explanation?performances=1,0.5,1,0.5&other=true HTTP/1.1" 200 -
